# Weekly Challenge 15

*Original URL* https://community.alteryx.com/t5/Weekly-Challenge/Challenge-15-Warehouse-Shipped-Miles/td-p/36744 and [**My Alteryx Approach**](https://github.com/dsmdavid/Alteryx-Weekly-Challenge/tree/master/submitted/sub_Challenge%2315)

## Brief
Based on last week’s warehouse distribution exercise, we want to calculate the total shipped miles per item.  The products are available from 3 different warehouses. 

The objective is to find the total distance travelled (straight line miles) for each item based on it being shipped from the closest warehouse.  


In [58]:
import pandas as pd
import os
import numpy as np
from geopy.distance import geodesic, great_circle, vincenty

## Approach I want to follow:
1. Read the files, find the closest warehouse to each location.
1. Multiply the distance times the number of items.
1. Summarize.

## 1. Read and combine

In [2]:
#Read the files
os.chdir(os.path.join(os.getcwd(), '15_files'))

In [3]:
os.listdir()

['01_storesPriority.csv', '02_assignedItems.csv', 'warehouseLocations.csv']

In [47]:
#Load the data:
input_dfStores = pd.read_csv("./01_storesPriority.csv", encoding="latin")     
input_dfItems = pd.read_csv("./02_assignedItems.csv", encoding = "latin")
input_dfWarehouse = pd.read_csv("./warehouseLocations.csv", encoding = "latin")

In [48]:
#Get LatLon points for distances:
input_dfWarehouse['loc'] = input_dfWarehouse.apply(lambda row: (row['Lat'], row['Lon']), axis =1)

input_dfStores['loc'] = input_dfStores.apply(lambda row: (row['Lat'], row['Lon']), axis =1)

In [49]:
input_dfWarehouse

,Warehouse,City,State,Lat,Lon,loc
0,Main,Huntsville,AL,34.737177,-86.603266,"(34.737177, -86.603266)"
1,Second,El Paso,TX,31.767002,-106.492058,"(31.767002, -106.49205800000001)"
2,Third,Colorado Springs,CO,38.825775,-104.831478,"(38.825775, -104.831478)"


In [50]:
temp = input_dfWarehouse[['Warehouse','loc']].transpose()
temp.columns = temp.loc['Warehouse',:]
temp.drop(labels='Warehouse', inplace=True)
temp['join'] = 1
temp

Warehouse,Main,Second,Third,join
loc,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",1


In [51]:
input_dfStores['join'] = 1

In [52]:
new_df = input_dfStores.merge(temp, on='join')
new_df.reset_index(drop=True, inplace=True)

In [59]:
def calc_distance(point1,point2):
    return vincenty(point1,point2).miles
    return geodesic(point1,point2).miles
    return great_circle(point1, point2).miles
# geodesic(point1,point2).miles

In [35]:
def shorter_distance_to_warehouse(row):
    return min([calc_distance(row['loc'],row[name]) for name in ["Main","Second", "Third"]])

In [60]:
new_df['shorter_distance'] = new_df.apply(shorter_distance_to_warehouse, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [61]:
new_df

,Unnamed: 0,Store,Priority,City,State,Lon,Lat,loc,join,Main,Second,Third,shorter_distance
0,0,A,26,Houston,TX,-95.562207,29.839187,"(29.839187, -95.562207)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",623.173245
1,1,B,4,Birmingham,AL,-86.625591,33.410321,"(33.410321, -86.625591)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",91.462267
2,2,C,25,Seattle,WA,-122.291399,47.603993,"(47.603993, -122.291399)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",1065.817190
3,3,D,3,Detroit,MI,-83.255088,42.412144,"(42.412144, -83.255088)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",559.448123
4,4,E,5,Nashville,TN,-86.729174,36.135464,"(36.135464, -86.729174)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",96.659644
5,5,F,2,Chicago,IL,-87.597287,41.734000,"(41.734, -87.597287)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",485.601345
6,6,G,24,Lansing,IA,-91.228717,43.361018,"(43.361018, -91.228717)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",644.547762
7,7,H,1,Fayetteville,NC,-78.907738,35.024302,"(35.024302, -78.907738)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",437.491536
8,8,I,23,Miami,FL,-80.310470,25.792304,"(25.792304, -80.31047)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",721.535424
9,9,J,22,Tampa,FL,-82.539427,27.886595,"(27.886595, -82.539427)",1,"(34.737177, -86.603266)","(31.767002, -106.49205800000001)","(38.825775, -104.831478)",529.512214


In [42]:
new_df = new_df[['Store', 'shorter_distance']]

In [43]:
new_df

,Store,shorter_distance
0,A,622.761638
1,B,91.685923
2,C,1064.189102
3,D,560.152623
4,E,96.872054
5,F,486.424399
6,G,645.196704
7,H,436.528046
8,I,722.752613
9,J,530.506043


In [44]:
distances_df = input_dfItems.merge(new_df, on ='Store')

In [45]:
distances_df['sumPerStore'] = distances_df['Assigned']*distances_df['shorter_distance']

In [46]:
distances_df.groupby(by='Item').sum()['sumPerStore']

Item
1     468024.289740
2     521239.887586
3     418762.042235
4     471745.363191
5     613430.597930
6     338620.836233
7     654526.543649
8     465329.017342
9     385656.099113
10    385975.918792
Name: sumPerStore, dtype: float64

## 2. Create functions

In [7]:
def shipping(row):
    if row['Required'] <= row['Count']:
        return row['Required']
    else:
        return row['Count']
    

In [8]:
def fulfill_store(store_str, input_dfWarehouse):
    #Process one store:
    #1. Get items required
    dfItems = input_dfRequirements[input_dfRequirements['Store']==store_str][['Item','Required']]
    dfItems.set_index('Item', inplace=True)
    #2. Check against warehouse
    dfTransient = input_dfWarehouse.join(dfItems, how='left').fillna(0)
    #3. Get the min(required, in warehouse)
    dfTransient['Assigned'] = dfTransient.apply(lambda row:shipping(row), axis=1)
    #4. Return the store.
    dfStore = dfTransient.copy()[['Required','Assigned','Warehouse']]
    dfStore['Store'] = store_str
    #5. Return the updated warehouse
    dfTransient['Count'] = dfTransient['Count'] - dfTransient['Assigned']
    input_dfWarehouse = dfTransient.drop(labels =['Required', 'Assigned'],axis=1).copy()

    return dfStore, input_dfWarehouse


In [9]:
#Starting Warehouse
input_dfWarehouse

,Warehouse,Count
Item,,
1,Main,824
2,Main,1145
3,Main,1354
4,Main,916
5,Main,1120
6,Main,635
7,Main,1056
8,Main,824
9,Main,616


## 3. Process stores

In [10]:
#Process all stores
list_df = []
for i in storesInOrder:
    new_store, input_dfWarehouse = fulfill_store(store_str = i, input_dfWarehouse = input_dfWarehouse)
    list_df.append(new_store)
    

In [11]:
#ending warehouse
input_dfWarehouse

,Warehouse,Count
Item,,
1,Main,73.0
2,Main,288.0
3,Main,662.0
4,Main,159.0
5,Main,110.0
6,Main,0.0
7,Main,0.0
8,Main,0.0
9,Main,0.0


## 4. Compare to existing results

In [12]:

#Load existing results
solution_df = pd.read_csv("./04_solution.csv", encoding = "latin")

In [13]:
# At this point, the results exist as a dataframe per store with all the potential items with the corresponding assignments.
# The output to compare against is a single dataframe with all stores with only the items that were required

#1. Concatenate all the stores
updated_stores = pd.concat(list_df,axis=0)

updated_stores['Item'] = updated_stores.index

output_df = input_dfStores[['Store','Priority','City','State']].merge(updated_stores, left_on='Store', right_on='Store')

#2. Keep only items with a requirement, change the order of the columns to match the order in the solutions and match the dtypes
test = output_df[output_df['Required']!=0][solution_df.columns].astype({'Required':'int64', 'Assigned':'int64'})
test.reset_index(drop=True, inplace=True)

In [14]:
test.equals(solution_df)

True